In [1]:
## Importing necessary libraries
import sys
import sqlite3
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

# defining functions module path
module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import normalize, clean_data

In [2]:
# Getting the transactions data from sqlite
conn = sqlite3.connect("../data/bootcamp_db") # creating connection

In [3]:
# Quering the data from the online_transactions_fixed table
query = """
select *
from online_transactions_fixed
"""
ot = pd.read_sql(query, conn)

In [4]:
# The function clean_data performed a serie of cleaning steps defined on EDA notebook
online_trans = clean_data(ot)

In [5]:
online_trans.describe()

,old_index,quantity,price,total_order_value,invoice_year,invoice_month,invoice_day,invoice_weekday
count,365180.000000,365180.000000,365180.000000,365180.000000,365180.000000,365180.000000,365180.000000,365180.000000
mean,201388.233230,12.879829,2.905252,21.919158,2010.934356,7.650090,15.011531,2.592034
std,115651.048694,43.577928,7.381186,95.735167,0.247660,3.405954,8.635549,1.920402
min,0.000000,1.000000,0.001000,0.001000,2010.000000,1.000000,1.000000,0.000000
25%,101147.750000,2.000000,1.250000,4.950000,2011.000000,5.000000,7.000000,1.000000
50%,202427.500000,6.000000,1.950000,11.900000,2011.000000,8.000000,15.000000,2.000000
75%,301305.250000,12.000000,3.750000,19.800000,2011.000000,11.000000,22.000000,4.000000
max,399806.000000,4800.000000,1599.260000,38970.000000,2011.000000,12.000000,31.000000,6.000000


#### In the previous notebook (EDA.ipynb) the data was grouped by customer id and the values of Recency, Frequency, and Monetary Value were computed. 
- Recency was defined as the difference between the current day and the day of the customer's last purchase. As the data is from 2011, and a customer whose recency is bigger than a year can be considered as churned, I chosed the "current day" as the last day of the last purchase on the data set, meaning that the minimum value that recency can take is 0.
- Frequency was define as the number of transactions that a customer performed.
- Monetary Value is simply the sum of the total order value per customer on the dataset

In this notebook, I am going to implement a customer segmentation analysis focusing on an percentile-based grouping RFM score

In [6]:
# Importing customer data
cust_data = pd.read_csv("../data/customer_data.csv")

In [7]:
cust_data.describe()

,loyalty_time,mean_nstock,mean_nitem,mon_value,frequency,recency
count,4247.000000,4247.000000,4247.000000,4247.000000,4247.000000,4247.000000
mean,133.676948,21.560731,224.903868,1884.727573,85.985401,88.463386
std,132.639094,19.285931,305.002619,8210.321624,215.913912,98.735861
min,0.000000,1.000000,1.000000,0.850000,1.000000,0.000000
25%,0.000000,9.041667,90.583333,297.200000,16.000000,15.500000
50%,101.000000,16.500000,158.235294,635.680000,39.000000,46.000000
75%,257.000000,28.000000,265.483333,1534.090000,94.000000,134.000000
max,372.000000,300.647059,7824.000000,279138.020000,7499.000000,373.000000


### Let's calculated the RFM score based on a percentile grouping

In [8]:
r_quartiles = pd.qcut(cust_data['mon_value'], q=4, labels=range(4, 0, -1))
f_quartiles = pd.qcut(cust_data['frequency'], q=4, labels=range(1,5))
m_quartiles = pd.qcut(cust_data['mon_value'], q=4, labels=range(1,5))

In [9]:
cust_data['R'] = r_quartiles
cust_data['F'] = f_quartiles
cust_data['M'] = m_quartiles

In [10]:
def rfm_segm(x):
    return str(x['R']) + str(x['F']) + str(x['M'])
cust_data['RFM_segment'] = cust_data.apply(rfm_segm, axis=1) 

In [11]:
cust_data['RFM_score'] = cust_data[['R', 'F', 'M']].sum(axis=1)

In [12]:
cust_data.sort_values("RFM_segment")

,customer_id,loyalty_time,mean_nstock,mean_nitem,mon_value,frequency,recency,R,F,M,RFM_segment,RFM_score
3325,u16986,0,1.500000,520.000000,1873.20,3.0,29,1,1,4,114,6
3113,u16698,26,2.500000,420.000000,1998.00,5.0,226,1,1,4,114,6
3094,u16671,136,2.000000,291.666667,1692.27,6.0,27,1,1,4,114,6
3550,u17317,43,4.000000,555.000000,2562.18,8.0,45,1,1,4,114,6
2647,u16041,110,2.000000,528.000000,3166.40,6.0,30,1,1,4,114,6
...,...,...,...,...,...,...,...,...,...,...,...,...
3730,u17561,0,45.000000,144.000000,165.12,45.0,18,4,3,1,431,8
2563,u1592,0,19.333333,31.333333,163.55,58.0,153,4,3,1,431,8
2537,u15877,16,106.000000,172.000000,239.31,106.0,1,4,4,1,441,9
1937,u1506,0,28.500000,64.000000,293.00,114.0,7,4,4,1,441,9


In [17]:
# Let's inspect number of customers in each segment
cust_data.groupby('RFM_segment').size().sort_values(ascending=False)

RFM_segment
144    767
411    724
322    553
233    542
421    242
312    234
243    228
223    217
332    212
134    207
431     93
213     74
342     63
124     57
114     31
441      3
dtype: int64

In [20]:
cust_data.groupby('RFM_segment').get_group('441')

,customer_id,loyalty_time,mean_nstock,mean_nitem,mon_value,frequency,recency,R,F,M,RFM_segment,RFM_score
1937,u1506,0,28.5,64.0,293.00,114.0,7,4,4,1,441,9
2537,u15877,16,106.0,172.0,239.31,106.0,1,4,4,1,441,9
3512,u17254,10,55.5,125.5,271.19,111.0,4,4,4,1,441,9


In [24]:
cust_data['RFM_score'].describe()

count    4247.000000
mean        7.497528
std         1.118268
min         6.000000
25%         6.000000
50%         7.000000
75%         8.000000
max         9.000000
Name: RFM_score, dtype: float64